In [415]:
import numpy as np
import pandas as pd
from imutils import paths
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [316]:
dataPaths = list(paths.list_files('.\ipl_csv2'))
dataPaths.pop(len(dataPaths) - 1)
dataPaths.pop(len(dataPaths) - 2)



def retrieve_past_data(team_a, team_b, num):
    req_df = []
    # stadium_avg = {}
    # unique_bowlers = set()
    # unique_batsmen = set()

    for match_file in dataPaths:

        try:
            df = pd.read_csv(match_file, index_col='match_id')
            # temp_a, temp_b = get_unique_players(df)
            # unique_bowlers |= (temp_a) 
            # unique_batsmen |= (temp_b) 
            df = preprocess(df)
            # stadium_avg = get_stadium_avg(stadium_avg, df)
            if (df.iloc[1]['batting_team'] == team_a or df.iloc[1]['batting_team'] == team_b) and (df.iloc[1]['bowling_team'] == team_a or df.iloc[1]['bowling_team'] == team_b):
                if len(req_df) < num or num == -1:
                    req_df.append(df)
        
        except Exception as e:
            print(e)

    return req_df

def preprocess(df):
    pp_df = df[df['ball'] <= 5.6]
    pp_df = pp_df[pp_df['innings'] <= 2]

    pp_df['total_runs'] = pp_df['runs_off_bat'] + pp_df['extras']

    return pp_df

def get_stadium_avg(stad_dict, df):
    venue = df.iloc[1]['venue']
    runs1 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 1)]['total_runs'].sum()
    runs2 = df.loc[(df['ball'] < 6.1) & (df['innings'] == 2)]['total_runs'].sum()

    if (runs1 <= 150 and runs1 >= 10):
        stad_dict.setdefault(venue,[]).append(runs1)
    if (runs2 <= 150 and runs2 >= 10):
        stad_dict.setdefault(venue,[]).append(runs2)

    return stad_dict

# def get_unique_players(df):
#     bowlers = set(df['bowler'].unique().tolist())
#     batsmen_striker = set(df['striker'].unique().tolist())
#     batsmen_non_striker = set(df['non_striker'].unique().tolist())
    
#     batsmen = batsmen_striker | (batsmen_non_striker)

#     return bowlers, batsmen


In [317]:
req_df = retrieve_past_data("Chennai Super Kings", "Kings XI Punjab", -1)
# stad_avg['M.Chinnaswamy Stadium'].extend(stad_avg['M Chinnaswamy Stadium'])
# stad_avg.pop('M Chinnaswamy Stadium')

# stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk'])
# stad_avg['MA Chidambaram Stadium'].extend(stad_avg['MA Chidambaram Stadium, Chepauk, Chennai'])
# stad_avg.pop('MA Chidambaram Stadium, Chepauk')
# stad_avg.pop('MA Chidambaram Stadium, Chepauk, Chennai')

# stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association IS Bindra Stadium, Mohali'])
# stad_avg['Punjab Cricket Association IS Bindra Stadium'].extend(stad_avg['Punjab Cricket Association Stadium, Mohali'])
# stad_avg.pop('Punjab Cricket Association IS Bindra Stadium, Mohali')
# stad_avg.pop('Punjab Cricket Association Stadium, Mohali')

# stad_avg['Rajiv Gandhi International Stadium'].extend(stad_avg['Rajiv Gandhi International Stadium, Uppal'])
# stad_avg.pop('Rajiv Gandhi International Stadium, Uppal')

# stad_avg['Arun Jaitley Stadium'].extend(stad_avg['Feroz Shah Kotla'])
# stad_avg.pop('Feroz Shah Kotla')

# stad_avg['Wankhede Stadium'].extend(stad_avg['Wankhede Stadium, Mumbai'])
# stad_avg.pop('Wankhede Stadium, Mumbai')

# print(stad_avg)

C:\Users\rishi\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3357: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [294]:

# # Importing library
# import csv
  
# # data to be written row-wise in csv fil
# data = list(u_batsmen)
# data.sort()

# with open('batsmen.csv', 'w') as myfile:
#      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

#      for d in data:
#         wr.writerow([d])

# print(len(data))

544


In [318]:
# stad_names = list(stad_avg.keys())
# keys = pd.Series(stad_names, dtype="category")
# vals = [float(len(stad_avg[k])) for k in keys.cat.categories]


# plt.figure(figsize=(20, 10))
# sns.barplot(x=keys.cat.codes, y=vals)

In [319]:
# dict(enumerate(keys.cat.categories))

In [395]:
test = req_df[len(req_df) - 1]
train_set = req_df[:len(req_df) - 1]

test_batsmen = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))

test_bowlers = test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['bowler'].unique()

test_x = pd.DataFrame({'venue': [test.iloc[2]['venue']], 'innings': [1], 'batting_team': [test.iloc[2]['batting_team']], 'bowling_team':[test.iloc[2]['bowling_team']], 'batsmen': [test_batsmen], 'bowlers': [test_bowlers]})

team_a = test.iloc[1]['batting_team']
team_b = test.iloc[1]['bowling_team']

test_x.head()

for df in train_set:
    match_id = df.index[0]

    inn = df[(df['batting_team'] == team_a) & (df['bowling_team'] == team_b)]['innings'].iloc[0] #innings for which we are going to predict

    runs_x = df.loc[(df['ball'] < 6.1) & (df['innings'] == inn)]['total_runs'].sum() #runs of first innings
    runs_target = df.loc[(df['ball'] < 6.1) & (df['innings'] == 3 - inn)]['total_runs'].sum() #runs of second innings
    
    #MODEL CODE GOES HERE
    train_x = pd.DataFrame('venue': [get_venue_score(df)], 'batsmen_rating': [get_batsmen_scores(df)], 'bowlers_rating': [get_bowlers_scores(df)], 'wickets': [get_wickets(df)])

In [ ]:
def get_venue_score(df):
    venue = df.iloc[1]['venue']
    #assign venue scores here

    return random.uniform(0, 1)

def get_batsmen_scores(df_list):
    u_bat = list(set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['striker'].unique()) | set(test.loc[(test['ball'] < 6.1) & (test['innings'] == 1)]['non_striker'].unique()))

    for df in df_list:
        
    
    return random.uniform(0, 1)

In [406]:
def init_params(layer_dims):
    parameters = {}
    for l in range(1, len(layer_dims)):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.1
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

In [407]:
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

def relu(Z):
    A = np.maximum(0,Z)
    cache = Z 
    
    return A, cache

def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

def linear_activation_forward(A_prev, W, b, activation):
    
    Z, linear_cache = linear_forward(A_prev, W, b)
    
    if (activation == 'relu'):
        A, activation_cache = relu(Z)
    elif (activation == 'sigmoid'):
        A, activation_cache = sigmoid(Z)
        
    cache = (linear_cache, activation_cache)

    return A, cache

def L_layer_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(1, L):
        A, cache = linear_activation_forward(A, parameters["W" + str(l)], parameters["b" + str(l)], "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")
    caches.append(cache)
         
    return AL, caches

In [408]:
def compute_cost(AL, Y):
    m = Y.shape[1]

    cost = -np.sum((Y*np.log(AL) + (1 - Y)*np.log(1 - AL)) / m)
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    
    return cost

In [412]:

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True) 
    
    dZ[Z <= 0] = 0

    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
        
    return dZ


def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = np.dot(dZ,A_prev.T)/m
    db = np.sum(dZ, axis = 1, keepdims = True) / m
    dA_prev = np.dot(W.T,dZ)
    
    return dA_prev, dW, db


def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    
    return dA_prev, dW, db


def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    current_cache = caches[L - 1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp


    return grads


In [413]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate*grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate*grads["db" + str(l+1)]
    return parameters

In [414]:
nx = train_x.shape[0]
ny = 1
layers_dims = [nx, ny]

def L_layer_model(X, Y, layers_dims, parameters = init_params(layers_dims), learning_rate = 0.5, num_iterations = 5000, print_cost=False):#lr was 0.009

    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)
    
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_layer_forward(X, parameters)
        
        # Compute cost.
        cost = compute_cost(AL, Y)
    
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
 
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)

                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

NameError: name 'layers_dims' is not defined